## 🤔 Config

In [ ]:
#|default_exp utils.config

In [ ]:
# |hide
from nbdev.showdoc import *

In [ ]:
# |hide
# |export
from copy import copy
from types import SimpleNamespace
from contextlib import contextmanager

import numpy as np

In [ ]:
# |exporti
_config = SimpleNamespace(
    precision     = 3,    # Digits after `.`
    threshold_max = 3,    # .abs() larger than 1e3 -> Sci mode
    threshold_min = -4,    # .abs() smaller that 1e-4 -> Sci mode
    sci_mode      = None, # None = auto. Otherwise, force sci mode (2.3e4) on/off.
    indent        = 2,    # Indent for .deeper()
    color         = True, # ANSI colors in text
)

In [ ]:
# |export
def set_config( precision       :int =None,     # Digits after `.`
                threshold_min   :int =None,     # .abs() larger than 1e3 -> Sci mode
                threshold_max   :int =None,     # .abs() smaller that 1e-4 -> Sci mode
                sci_mode        :bool=None,     # None = auto. Otherwise, force sci mode (2.3e4) on/off.
                indent          :bool=None,     # Indent for .deeper()
                color           :bool=None):    # ANSI colors in text

    "Set config variables"
    for k,v in locals().items():
        if v is not None: setattr(_config, k, v)

In [ ]:
# |export
def get_config():
    "Get a copy of config variables"
    return copy(_config)

In [ ]:
# |export
@contextmanager
def config( precision       :int =None,     # Digits after `.`
            threshold_min   :int =None,     # .abs() larger than 1e3 -> Sci mode
            threshold_max   :int =None,     # .abs() smaller that 1e-4 -> Sci mode
            sci_mode        :bool=None,     # None = auto. Otherwise, force sci mode (2.3e4) on/off.
            indent          :bool=None,     # Indent for .deeper()
            color           :bool=None):    # ANSI colors in text
    "Context manager for temporarily setting printting options."
    global _config
    new_opts = locals()
    opts = get_config()

    try:
        set_config(**new_opts)
        yield
    finally:
        _config = opts

In [ ]:
show_doc(set_config)

---

[source](https://github.com/xl0/lovely-numpy/blob/master/lovely_numpy/utils/config.py#L24){target="_blank" style="float:right; font-size:smaller"}

### set_config

>      set_config (precision:int=None, threshold_min:int=None,
>                  threshold_max:int=None, sci_mode:bool=None, indent:bool=None,
>                  color:bool=None)

Set config variables

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| precision | int | None | Digits after `.` |
| threshold_min | int | None | .abs() larger than 1e3 -> Sci mode |
| threshold_max | int | None | .abs() smaller that 1e-4 -> Sci mode |
| sci_mode | bool | None | None = auto. Otherwise, force sci mode (2.3e4) on/off. |
| indent | bool | None | Indent for .deeper() |
| color | bool | None | ANSI colors in text |

In [ ]:
show_doc(get_config)

---

[source](https://github.com/xl0/lovely-numpy/blob/master/lovely_numpy/utils/config.py#L36){target="_blank" style="float:right; font-size:smaller"}

### get_config

>      get_config ()

Get a copy of config variables

In [ ]:
show_doc(config)

---

[source](https://github.com/xl0/lovely-numpy/blob/master/lovely_numpy/utils/config.py#L42){target="_blank" style="float:right; font-size:smaller"}

### config

>      config (precision:int=None, threshold_min:int=None,
>              threshold_max:int=None, sci_mode:bool=None, indent:bool=None,
>              color:bool=None)

Context manager for temporarily setting printting options.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| precision | int | None | Digits after `.` |
| threshold_min | int | None | .abs() larger than 1e3 -> Sci mode |
| threshold_max | int | None | .abs() smaller that 1e-4 -> Sci mode |
| sci_mode | bool | None | None = auto. Otherwise, force sci mode (2.3e4) on/off. |
| indent | bool | None | Indent for .deeper() |
| color | bool | None | ANSI colors in text |

### Examples

In [ ]:
from lovely_numpy import Lo, set_config, get_config, config

In [ ]:
set_config(precision=5)

Lo(np.array([1., 2, 3]))

array[3] x∈[1.00000, 3.00000] μ=2.00000 σ=0.81650 [1.00000, 2.00000, 3.00000]

In [ ]:
with config(sci_mode=True):    
    print(Lo(np.array([1., 2, 3])))

array[3] x∈[1.00000e+00, 3.00000e+00] μ=2.00000e+00 σ=8.16497e-01 [1.00000e+00, 2.00000e+00, 3.00000e+00]


In [ ]:
set_config(precision=3)
Lo(np.array([1., 2, 3]))

array[3] x∈[1.000, 3.000] μ=2.000 σ=0.816 [1.000, 2.000, 3.000]

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()